In [10]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [11]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_52181/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [13]:
# 测试下to_csv函数
all_data.iloc[:10].to_csv('all_data_10.csv')
all_data_10 = pd.read_csv('all_data_10.csv')
all_data_10[['cv_id', 'eduTracks']]

,cv_id,eduTracks
0,2c9207157bf91042017c0d2beef00470,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":..."
1,2c9207157bf91042017c0d979cbb047d,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":..."
2,2c9207157bf91042017c7232fbab3be1,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":..."
3,2c9207157d308592017d4a94cb1113a1,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":..."
4,2c9207157dabb0a3017dbca16cdf4c26,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":..."
5,2c9207157dbedf8d017dc1503c891402,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":..."
6,2c9207157dbedf8d017dc1503c891402,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":..."
7,2c9207157dbedf8d017dc651bdcf5638,"['{""degree"": ""本科"", ""endDate"": ""2017-01"", ""id"":..."
8,2c9207157dbedf8d017dc651bdcf5638,"['{""degree"": ""本科"", ""endDate"": ""2017-01"", ""id"":..."
9,2c9207157dbedf8d017dc7561a247ced,"['{""degree"": ""大专"", ""endDate"": ""2015-06"", ""id"":..."


In [5]:
all_data[['work_age', 'workYear']]

,work_age,workYear
0,5-10年,13.0
1,3-5年,7.0
2,10年以上,14.0
3,3-5年,3.0
4,3-5年,5.0
...,...,...
81291,5-10年,1.0
81292,10年以上,23.0
81293,10年以上,23.0
81294,3-5年,5.0


## 理清需要用的特征


In [9]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 'pay_months', 'currentSalary', 'desiredSalary']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_words', 'equal_job']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [11]:
all_data[['cv_id', 'jd_id']+raw_columns]

,cv_id,jd_id,age,recruit_number,workYear,min_annual_salary,max_annual_salary,pay_months,currentSalary,desiredSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,80.0,150.0,NaN,NaN,NaN
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,28.0,35.0,NaN,NaN,NaN
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,25.0,35.0,NaN,NaN,NaN
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,60.0,500.0,NaN,NaN,NaN
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,NaN,NaN,12.0,保密,25 - 30k · 15薪
...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,41.0,1,1.0,NaN,NaN,12.0,6000,6千-8千/月
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,47.0,1,23.0,NaN,NaN,12.0,60+期权,100
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,47.0,1,23.0,NaN,NaN,13.0,60+期权,100
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,30.0,30,5.0,NaN,NaN,14.0,25K,28K


## tfidf_pca

In [1]:
import pandas as pd
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_56824/358587841.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
sample = all_data

# 需要将这个向量变成一个列表
def tfidf_pca_merge_fun(series):
    return list(series)

tfidf_pca_file = ['description_tfidf_pca.csv', 'position_tfidf_pca.csv', 'requirement_tfidf_pca.csv', 'skills_tfidf_pca.csv', 'title_category_tags_tfidf_pca.csv']
for csv_file in tfidf_pca_file:
    tfidf_pca = pd.read_csv('../data'+csv_file).drop(['Unnamed: 0'], axis=1)
    print(csv_file)
    #print(tfidf_pca)
    col_name = csv_file.strip('.csv')
    tfidf_pca[col_name] = tfidf_pca.apply(tfidf_pca_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_pca[[col_name]]], axis=1)

tfidf_pca_col_names = [csv_file.strip('.csv') for csv_file in tfidf_pca_file]
sample[tfidf_pca_col_names]

description_tfidf_pca.csv
position_tfidf_pca.csv
requirement_tfidf_pca.csv
skills_tfidf_pca.csv
title_category_tags_tfidf_pca.csv


,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca
0,"[-0.2036062295725057, 0.0482743259611426, 0.06...","[-0.0393278199932224, -0.1156517780600516, -0....","[-0.0884550054331397, -0.125519552436948, -0.0...","[0.3299248867367418, 0.164741969808482, 0.1889...","[-0.0428003378972316, -0.1584212552141981, -0...."
1,"[-0.0922193627558096, 0.0553739463988321, -0.1...","[0.1054180773106984, -0.0263971997890037, -0.0...","[-0.0803609394513418, -0.1239125350185267, -0....","[0.2365059699800738, 0.1136815262526436, -0.23...","[-0.2368437904582563, 0.0858237305581985, -0.2..."
2,"[-0.0731212086793756, -0.1819505396457883, -0....","[-0.0368150744980858, -0.0868620766763974, -0....","[-0.0644952418134994, -0.1617720130418969, -0....","[-0.0915394250400994, -0.1409954987383418, -0....","[-0.1908741398655453, 0.1941297757385749, 0.34..."
3,"[-0.0941188808553261, 0.0989578513909629, -0.0...","[0.0490419846368333, -0.0412213764571144, -0.0...","[-0.1351532040885177, 0.1740611212306139, 0.04...","[0.1429876436797454, 0.0219408706921027, -0.14...","[-0.3720983915852013, 0.191217992346987, -0.31..."
4,"[0.0394442348392252, -0.0330071666173531, 0.06...","[-0.0271020068507462, -0.1132012422332272, -0....","[-0.0138174200364112, 0.1003272366963715, -0.0...","[-0.0992961440476657, -0.1495964296220377, -0....","[0.0056217778664832, -0.0571649606124799, 0.00..."
...,...,...,...,...,...
81291,"[-0.0614540249096187, -0.0867970356720048, -0....","[-0.0745961609163453, -0.052911867138294, -0.0...","[-0.0400398533506595, -0.0616278716402495, 0.0...","[-0.0986273352712469, -0.1820012999649072, -0....","[0.0344861435113101, -0.0468051330650081, 0.05..."
81292,"[-0.1197626313820459, 0.2009878509549596, 0.10...","[-0.0248770950143107, -0.0818758151299455, -0....","[-0.1178366614102289, 0.125663028721799, 0.141...","[0.0156513997900681, 0.0238285941465905, -0.06...","[0.0151640498218406, -0.0802995831055218, 0.00..."
81293,"[-0.1197626313820459, 0.2009878509549596, 0.10...","[-0.0248770950143107, -0.0818758151299455, -0....","[-0.1178366614102289, 0.125663028721799, 0.141...","[0.0156513997900681, 0.0238285941465905, -0.06...","[-0.0053327153389531, -0.0845578727706244, 0.0..."
81294,"[0.4074295021733307, -0.0564815548542076, 0.38...","[-0.3849339451860117, 0.4677351578570149, 0.34...","[0.533978782672698, 0.0783524240538382, 0.1293...","[-0.2529650764970855, 0.1589143672355407, 0.08...","[0.547197635656256, 0.4049100240667704, -0.158..."


## 数据集划分，剔除复推集